In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
import json
import time
import os
from rtsvg import *
rt = RACETrack()

_base_ = '../../../data/kaggle_imdb_600k/international-movies-json/'
_files_ = os.listdir(_base_)
print(f'Loading {len(_files_)} files...')
_jsons_ = []
for i in range(len(_files_)):
    _file_ = _files_[i]
    _txt_  = open(_base_ + _file_).read()
    _json_ = json.loads(_txt_)
    _jsons_.append(_json_)

print('Applying template to json...')

_xform_map_ = '''
__id__              = '$[*]._id'              | MovieID      | uniq
__director__        = '$[*].director.name_id' | DirectorID   | uniq
__director__        --- "directedMovie"  --- __id__ ^^^ "imdb_600k_international_movies"
'''

_code_example_ = '''
for i in range(len(myjson)):
    if "_id" in myjson[i]:
        _var1_ = myjson[i]["_id"]
    else: _var1_ = None
    if "director" in myjson[i]:
        if "name_id" in myjson[i]["director"]:
            _var0_ = myjson[i]["director"]["name_id"]
        else: _var0_ = None
        filled["$[*].director.name_id"].append(_var0_)
        filled["$[*]._id"].append(_var1_)
'''

ofv_dir = rt.ontologyFrameworkInstance(xform_spec=_xform_map_)
ofv_dir.parse(_jsons_)
#ofv_dir.df_triples

In [ ]:
_to_fill_        = ['$[*].director.name_id', '$[*]._id']
_filled_         = fillJSONPathElements(_to_fill_, _json_)
for _key_ in _filled_.keys(): print(f'{_key_=} {len(_filled_[_key_])}')

In [ ]:
_to_fill_        = ['$[*]._id']
_filled_         = fillJSONPathElements(_to_fill_, _json_)
for _key_ in _filled_.keys(): print(f'{_key_=} {len(_filled_[_key_])}')

In [ ]:
_movies_, _has_director_, missing_key_set = 0, 0, set()
for i in range(len(_json_)):
    if 'director' in _json_[i].keys():
        _has_director_ += 1
    else:
        missing_key_set.add(tuple(_json_[i].keys()))
    _movies_ += 1
print(missing_key_set)
_movies_, _has_director_